# Load necessary Python libraries

In [0]:
!nvidia-smi

Tue Dec 24 09:39:13 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
pip install SimpleITK

     |████████████████████████████████| 42.5MB 71kB/s 


In [0]:
pip install tensorflow-gpu=2.0.0

     |████████████████████████████████| 380.8MB 42kB/s 
     |████████████████████████████████| 450kB 54.1MB/s 
     |████████████████████████████████| 3.8MB 50.6MB/s 
     |████████████████████████████████| 81kB 14.2MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goo

# Mount your Google Drive

In [0]:
from google.colab import drive
drive.mount('M_drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at M_drive


# Import necessary libraries to environment

In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as itk
import matplotlib.animation as animation
from os.path import join
from PIL import Image  
import PIL  

%matplotlib notebook

import cv2
from sklearn.model_selection import train_test_split

# Check data in folders

In [0]:
path="M_drive/My Drive/SeminarAMS/"
DATA_PATH = path+'/synthetic_images'

# search all subfolders
images_path = os.listdir(DATA_PATH)
print(images_path)
print('V mapi {:s} je {:d} podmap.'.format(DATA_PATH, len(images_path)))


for i in range(0,len(images_path)):
    # load data from one folder
    map_path = join(DATA_PATH, images_path[i])#zaporedna številka mape

    im1 = itk.ReadImage(join(map_path,os.listdir(map_path)[0]))

    print('Ime mape: '+images_path[i])
    print('Število slik v mapi {}'.format(len(os.listdir(map_path))))
    print('Velikost posamezne slike je {}'.format(im1.GetSize()))
    print('Velikost vzorcenja {}'.format(im1.GetSpacing()))
    a=im1.GetSize()
    b=len(os.listdir(map_path))
    print('Število vseh slik izluščenih po vseh rezniah v mapi: {}'.format(a[0]*a[1]*a[2]*b))
    

['t1-images', 'flair-images', 't1-imagesLR', 't1-images-rawLR']
V mapi M_drive/My Drive/Colab Notebooks/Seminar_clean/SeminarSR_clean/images je 4 podmap.
Ime mape: t1-images
Število slik v mapi 28
Velikost posamezne slike je (352, 352, 165)
Velikost vzorcenja (0.6666666865348816, 0.6666666865348816, 0.9999973177909851)
Število vseh slik izluščenih po vseh rezniah v mapi: 572436480
Ime mape: flair-images
Število slik v mapi 28
Velikost posamezne slike je (240, 240, 321)
Velikost vzorcenja (1.0416666269302368, 1.0416666269302368, 0.5600041747093201)
Število vseh slik izluščenih po vseh rezniah v mapi: 517708800
Ime mape: t1-imagesLR
Število slik v mapi 36
Velikost posamezne slike je (352, 352, 165)
Velikost vzorcenja (0.6666666865348816, 0.6666666865348816, 0.9999973177909851)
Število vseh slik izluščenih po vseh rezniah v mapi: 735989760
Ime mape: t1-images-rawLR
Število slik v mapi 8
Velikost posamezne slike je (336, 336, 44)
Velikost vzorcenja (0.625, 0.625, 3.0000014305114746)
Števil

# Make HR images

In [0]:
path="M_drive/My Drive/SeminarAMS/"
DATA_PATH = path+'/synthetic_images'
save_path=path+"/dataset_MRI_synthetic/HR/"

# search all sub-folders
images_path = os.listdir(DATA_PATH)

save_id=0 #mark for saving
size=(252,252)

for i in range(0,1):
    # load data from one folder
    map_path = join(DATA_PATH, images_path[i])
    
    for j in range(0,len(os.listdir(map_path))):
        #load all images in folder
        im1 = itk.GetArrayFromImage(itk.ReadImage(join(map_path,os.listdir(map_path)[j])))
        Z,X,Y=im1.shape

        for z in range(20,Z-20,10):
            im=im1[z,:,:]
            im=cv2.resize(im, dsize=size, interpolation=cv2.INTER_CUBIC)
            im = (255.0 / im.max() * (im - im.min())).astype(np.uint8)
            im=Image.fromarray(im)
            im.save(save_path+str(save_id)+".png")
            save_id+=1
        
        for x in range(20,X-20,10):
            im=im1[:,x,:]
            im=cv2.resize(im, dsize=size, interpolation=cv2.INTER_CUBIC)
            im = (255.0 / im.max() * (im - im.min())).astype(np.uint8)
            im=Image.fromarray(im)
            im.save(save_path+str(save_id)+".png")
            save_id+=1
            
        for y in range(20,Y-20,10):
            im=im1[:,:,y]
            im=cv2.resize(im, dsize=size, interpolation=cv2.INTER_CUBIC)
            im = (255.0 / im.max() * (im - im.min())).astype(np.uint8)
            im=Image.fromarray(im)
            im.save(save_path+str(save_id)+".png")
            save_id+=1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in multiply


# Make LR images

In [0]:
import cv2
import numpy as np

path="M_drive/My Drive/SeminarAMS/"
DATA_PATH = path+'/dataset_MRI_synthetic/HR/'
save_path=path+"/dataset_MRI_synthetic/LR/"

images_path = os.listdir(DATA_PATH)
scale=3

for i in range(0,len(images_path)):
    if images_path[i].endswith('.png'):
        im1 = itk.GetArrayFromImage(itk.ReadImage(join(DATA_PATH,images_path[i])))
        X,Y=im1.shape
        x=np.round(X/scale).astype(np.uint16)
        y=np.round(Y/scale).astype(np.uint16)
        res = cv2.resize(im1, dsize=(x,y), interpolation=cv2.INTER_CUBIC)
        im=Image.fromarray(res)
        im.save(save_path+images_path[i])
    
    

In [0]:
path="M_drive/My Drive/SeminarAMS/"
DATA_PATH = path+'/dataset_MRI_synthetic/LR/'
images_path = os.listdir(DATA_PATH)
print(len(images_path))


DATA_PATH = path+'/dataset_MRI_synthetic/HR/'
images_path = os.listdir(DATA_PATH)
print(len(images_path))


2156
2156


# Shape all data into multidimensional array

In [0]:
from tqdm import tqdm
mri_data = []
path="M_drive/My Drive/SeminarAMS/"
LR_DATA_PATH = path+'/dataset_MRI_synthetic/LR/'
HR_DATA_PATH = path+'/dataset_MRI_synthetic/HR/'

lr_paths = os.listdir(LR_DATA_PATH)
for image_no in tqdm(range(len(lr_paths))):
    
    # read images
    hr = itk.ReadImage(join(HR_DATA_PATH, str(image_no)+'.png'))
    lr = itk.ReadImage(join(LR_DATA_PATH, str(image_no)+'.png'))

    mri_data.append({'hr':hr, 'lr':lr})

100%|██████████| 2156/2156 [00:48<00:00, 44.68it/s]


In [0]:
#join LR and HR images into 3D array
hr_array = np.dstack([np.squeeze(itk.GetArrayFromImage(data['hr'])) for data in mri_data])
lr_array = np.dstack([np.squeeze(itk.GetArrayFromImage(data['lr'])) for data in mri_data])
print('Velikost polja HR: {}'.format(hr_array.shape))
print('Velikost polja LR: {}'.format(lr_array.shape))

hr_array = np.transpose(hr_array, (2, 0, 1))
lr_array = np.transpose(lr_array, (2, 0, 1))
print('Velikost preoblikovanega polja HR: {}'.format(hr_array.shape))
print('Velikost preoblikovanega polja LR: {}'.format(lr_array.shape))

Velikost polja HR: (252, 252, 2156)
Velikost polja LR: (84, 84, 2156)
Velikost preoblikovanega polja HR: (2156, 252, 252)
Velikost preoblikovanega polja LR: (2156, 84, 84)


In [0]:
MODALITIES = ['hr', 'lr']

from keras import backend as K
print('Keras data format: {}'.format(K.image_data_format()))

# make data for training and validation
if K.image_data_format() == 'channels_first':
    hr_karray = hr_array[:, np.newaxis, :, :]
    lr_karray = lr_array[:, np.newaxis, :, :]
    channel_axis = 1
else:
    hr_karray = hr_array[:, :, :, np.newaxis]
    lr_karray = lr_array[:, :, :, np.newaxis]
    channel_axis = -1
print('Velikost polja HR slik {}'.format(hr_karray.shape))

    
print('Velikost polja LR slik {}'.format(lr_karray.shape))



Using TensorFlow backend.


Keras data format: channels_last
Velikost polja HR slik (2156, 252, 252, 1)
Velikost polja LR slik (2156, 84, 84, 1)


In [0]:
def multiply_dim(data,m,axis):
    '''
    Funkcija multiplicira željeno os v np.array

    Parameters
    ----------
    data : polje podatkov poljubne velikosti
    m : faktor množenja
    axis : številka osi ki jo želimo multiplicirati
    
    Returns
    --------
    odata : multiplicirani podatki

    '''
    
    odata=np.ndarray((data.shape[0],data.shape[1],data.shape[2],m))
    for i in range(m):
        odata[:,:,:,i]=np.squeeze(data,axis=axis)
    return odata

In [0]:
lr_data=multiply_dim(lr_karray,3,3)
hr_data=multiply_dim(hr_karray,3,3)
print(lr_data.shape)
print(hr_data.shape)

(2156, 84, 84, 3)
(2156, 252, 252, 3)


In [0]:

TEST_DATA_FRACTION = 0.2
X_train, X_test, y_train, y_test = train_test_split(lr_data[:2000,:,:,:], hr_data[:2000,:,:,:], test_size=TEST_DATA_FRACTION)

print('Velikost LR polja za učenje: {}'.format(X_train.shape))
print('Velikost LR polja za testiranje: {}'.format(X_test.shape))

print('Velikost HR polja za učenje: {}'.format(y_train.shape))
print('Velikost HR polja za testiranje: {}'.format(y_test.shape))

Velikost LR polja za učenje: (1600, 84, 84, 3)
Velikost LR polja za testiranje: (400, 84, 84, 3)
Velikost HR polja za učenje: (1600, 252, 252, 3)
Velikost HR polja za testiranje: (400, 252, 252, 3)


# Save training and validation data

In [0]:
path="M_drive/My Drive/SeminarAMS/"

save_dir=path+"/dataset_MRI_synthetic/dataset_split/X_train/"
os.makedirs(save_dir, exist_ok=True)
np.save(save_dir, X_train)

save_dir=path+"/dataset_MRI_synthetic/dataset_split/X_test/"
os.makedirs(save_dir, exist_ok=True)
np.save(save_dir, X_test)

save_dir=path+"/dataset_MRI_synthetic/dataset_split/y_train/"
os.makedirs(save_dir, exist_ok=True)
np.save(save_dir, y_train)

save_dir=path+"/dataset_MRI_synthetic/dataset_split/y_test/"
os.makedirs(save_dir, exist_ok=True)
np.save(save_dir, y_test)

FileNotFoundError: ignored